In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_dir="/content/drive/MyDrive/BTP/"

In [3]:
import os
import sys
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.activations import *
import matplotlib.pyplot as plt
from IPython import *
import cv2
from tqdm import *
import pickle
import random

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import *
from drive.MyDrive.BTP.utils import *

In [4]:
var_dtype='float32'

In [5]:
def im(q,ind,ext=".JPG"):
    return "q"+str(q)+"_im"+str(ind)+ext

def op(q,ind,ext=".JPG"):
    return "q"+str(q)+"_op"+str(ind)+ext

def getNames(q):
    return [im(q,i) for i in range(1,5)]+[op(q,i) for i in range(1,5)]

def imshow(img,scale=1):
    plt.imshow((scale*img).astype(np.uint8))
    plt.show()

def imread(fname,target_shape=None):
    img=cv2.imread(fname)
    if target_shape is not None: return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)
    return img

def resize(img,target_shape):
    return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)

In [6]:
def generator(x,y,bsize=32):
    if bsize==-1: yield x,y
    i=0
    n=x.shape[0]
    while i+bsize<=n:
        yield x[i:i+bsize],y[i:i+bsize]
        i+=bsize
    if i<n: yield x[i:],y[i:]

def generator_random(x,y,bsize=32):
    if bsize==-1: yield x,y
    else:
        n=x.shape[0]
        indexes=np.array([i for i in range(n)])
        np.random.shuffle(indexes)
        i=0
        while i+bsize<=n:
            yield x[indexes[i:i+bsize]],y[indexes[i:i+bsize]]
            i+=bsize
        if i<n: yield x[indexes[i:]],y[indexes[i:]]

In [7]:
root_data_dir=root_dir+"created_dataset/"

csv=root_dir+"image_cop.csv"

encoder_weights=root_dir+"trained_models/encoder.h5"
rel_decoder_weights=root_dir+"trained_models/rel_decoder.h5"

classifier_weights=root_dir+"trained_models/classifier.h5"
lstm_weights=root_dir+"trained_models/lstm.pkl"


lstm_units=128
inp_image_size=(64,64)
inp_image_channel=out_image_channel=3
inp_image_shape=inp_image_size+(inp_image_channel,)
batch_size=32
#rel_decoder_input_shape=(6,6,2048)

In [8]:
load_encoder_weights=root_dir+"trained_models/encoder.h5"
load_rel_decoder_weights=root_dir+"trained_models/rel_decoder.h5"
load_classifier_weights=root_dir+"trained_models/classifier.h5"
load_lstm_weights=root_dir+"trained_models/lstm.pkl"

In [20]:
train=(np.load(root_dir+'images_arrays.npy').astype(var_dtype))/255.0
#train=np.concatenate([np.load(root_dir+'test_images_arrays1.npy').astype(var_dtype)/255.0,np.load(root_dir+'test_images_arrays.npy').astype(var_dtype)/255.0,train],axis=0)
train=train.reshape((-1,)+inp_image_shape)
original=train.copy()
np.random.shuffle(train)
train.shape

(12000, 64, 64, 3)

In [21]:
y_train=train*255.0

In [11]:
encoder=getEncoder()

58900480/58889256 [==============================] - 2s 0us/step


In [12]:
rel_decoder=getDecoder(encoder)

In [13]:
#plot_model(rel_decoder,show_shapes=True)

In [14]:
def load_weights():
    encoder.load_weights(load_encoder_weights)
    rel_decoder.load_weights(load_rel_decoder_weights)


def save_weights():
    rel_decoder.save_weights(rel_decoder_weights)
    encoder.save_weights(encoder_weights)

#load_weights()

In [ ]:
# model=tk.models.Sequential([encoder,rel_decoder])
# model.compile(optimizer='rmsprop',loss=cusotmLoss)

In [ ]:
# model.fit(train[0:1],y_train[0:1],epochs=100)

In [15]:
def MSE(true_images,decoded_images):
    t=true_images-decoded_images
    t=tf.square(t)
    t=tf.reshape(t,(-1,))
    return tf.reduce_mean(t)

In [16]:
def train_decoder(x_train,opt,y_train):
    n=x_train.shape[0]
    zeros=tf.zeros((n,lstm_units),var_dtype)
    with tf.GradientTape(persistent=True) as tape:
        y=encoder(x_train)
        y=rel_decoder([zeros,y])
        loss=MSE(y_train,y)
    print(loss)
    grads1=tape.gradient(loss,rel_decoder.trainable_variables)
    grads2=tape.gradient(loss,encoder.trainable_variables)
    

    opt.apply_gradients(zip(grads1,rel_decoder.trainable_variables))
    opt.apply_gradients(zip(grads2,encoder.trainable_variables))

In [17]:
def test_decoder(x_train):
    #x_train=x_train.reshape((-1,)+inp_image_shape)
    #print(x_train.shape)
    #n=x_train.shape[0]
    n=x_train.shape[0]
    zeros=tf.zeros((n,lstm_units),var_dtype)
    y=encoder(x_train)
    y=rel_decoder([zeros,y]).numpy()
    #return np.clip((y+0.5)*255.0,0,255).astype(np.uint8)
    return np.clip(y,0,255).astype(np.uint8)

In [18]:
opt=Adam(0.001)#0.005

In [ ]:
epochs=20
opt.learning_rate=2.5e-5

for _ in range(epochs):
    print("Epoch {}:".format(_+1))
    for x,y in generator_random(train,y_train,250):
        train_decoder(x,opt,y_train=y)
        #break
    display.clear_output()
    #break

In [25]:
save_weights()
#load_weights()

In [ ]:
num=10
inds=np.random.randint(low=0,high=train.shape[0]-1,size=num)
imgs=original[inds]
#imgs=original[0:10]
gen=test_decoder(imgs)
fig = plt.figure(figsize=(20, 5))

for i in range(1, num+1):
    img = imgs[i-1]
    fig.add_subplot(2,num,i)
    plt.imshow(img)

    img = gen[i-1]
    fig.add_subplot(2,num,i+num)
    plt.imshow(img)

plt.show()

# imshow((test_decoder(original[i:i+1])[0]))
# imshow(original[i],scale=255)
# i

In [ ]:
original[0]